In [17]:
###Yield Management:

In [18]:
!pip install gurobipy



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: C:\Users\olw09\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [19]:
import gurobipy as gp
from gurobipy import GRB

# Define the lines for the tic-tac-toe constraints
lines = []
size = 3

for i in range(size):
    for j in range(size):
        for k in range(size):
            if i == 0:
                lines.append(((0,j,k), (1,j,k), (2,j,k)))
            if j == 0:
                lines.append(((i,0,k), (i,1,k), (i,2,k)))
            if k == 0:
                lines.append(((i,j,0), (i,j,1), (i,j,2)))
            if i == 0 and j == 0:
                lines.append(((0,0,k), (1,1,k), (2,2,k)))
            if i == 0 and j == 2:
                lines.append(((0,2,k), (1,1,k), (2,0,k)))
            if i == 0 and k == 0:
                lines.append(((0,j,0), (1,j,1), (2,j,2)))
            if i == 0 and k == 2:
                lines.append(((0,j,2), (1,j,1), (2,j,0)))
            if j == 0 and k == 0:
                lines.append(((i,0,0), (i,1,1), (i,2,2)))
            if j == 0 and k == 2:
                lines.append(((i,0,2), (i,1,1), (i,2,0)))

# Add the diagonal lines
lines.append(((0,0,0), (1,1,1), (2,2,2)))
lines.append(((2,0,0), (1,1,1), (0,2,2)))
lines.append(((0,2,0), (1,1,1), (2,0,2)))
lines.append(((0,0,2), (1,1,1), (2,2,0)))

print(len(lines))

model = gp.Model("tic-tac")

visit = model.addVars(size, size, size, vtype=GRB.BINARY, name='visit')
sameline = model.addVars(len(lines), vtype=GRB.BINARY, name='sameline')
model.addConstr(sum(visit[i,j,k] for i in range(size) for j in range(size) for k in range(size)) == 14)

for i, x in enumerate(lines):
    model.addConstr(sameline[i] >= visit[x[0]] + visit[x[1]] + visit[x[2]] - 2, name=f'constraint_{i}')
    model.addConstr(sameline[i] >= 1-(visit[x[0]] + visit[x[1]] + visit[x[2]]), name=f'constraint_{i}')

model.setObjective(sum(sameline[i] for i in range(len(lines))), GRB.MINIMIZE)

model.optimize()

if model.status == GRB.OPTIMAL:
    print('Solution:')
    print(f'Objective value = {model.objVal}')
else:
    print('No optimal solution found')


49
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 99 rows, 76 columns and 419 nonzeros
Model fingerprint: 0xf7452865
Variable types: 0 continuous, 76 integer (76 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 7.0000000
Presolve time: 0.00s
Presolved: 99 rows, 76 columns, 419 nonzeros
Variable types: 0 continuous, 76 integer (76 binary)

Root relaxation: objective 0.000000e+00, 40 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    8    7.00000   

In [20]:
import gurobipy as gp
from gurobipy import GRB

# Data
shifts = {
    "Mon1":  3,
    "Tue2":  2,
    "Wed3":  4,
    "Thu4":  4,
    "Fri5":  5,
    "Sat6":  6,
    "Sun7":  5,
    "Mon8":  2,
    "Tue9":  2,
    "Wed10": 3,
    "Thu11": 4,
    "Fri12": 6,
    "Sat13": 7,
    "Sun14": 5
}

shiftRequirements = {
    "Mon1":  3,
    "Tue2":  2,
    "Wed3":  4,
    "Thu4":  4,
    "Fri5":  5,
    "Sat6":  6,
    "Sun7":  5,
    "Mon8":  2,
    "Tue9":  2,
    "Wed10": 3,
    "Thu11": 4,
    "Fri12": 6,
    "Sat13": 7,
    "Sun14": 5
}

workers = {
    "Amy":   10,
    "Bob":   12,
    "Cathy": 10,
    "Dan":   8,
    "Ed":    8,
    "Fred":  9,
    "Gu":    11
}

pay = {
    "Amy":   10,
    "Bob":   12,
    "Cathy": 10,
    "Dan":   8,
    "Ed":    8,
    "Fred":  9,
    "Gu":    11
}

availability = [
    ('Amy', 'Tue2'), ('Amy', 'Wed3'), ('Amy', 'Fri5'), ('Amy', 'Sun7'),
    ('Amy', 'Tue9'), ('Amy', 'Wed10'), ('Amy', 'Thu11'), ('Amy', 'Fri12'),
    ('Amy', 'Sat13'), ('Amy', 'Sun14'), ('Bob', 'Mon1'), ('Bob', 'Tue2'),
    ('Bob', 'Fri5'), ('Bob', 'Sat6'), ('Bob', 'Mon8'), ('Bob', 'Thu11'),
    ('Bob', 'Sat13'), ('Cathy', 'Wed3'), ('Cathy', 'Thu4'), ('Cathy', 'Fri5'),
    ('Cathy', 'Sun7'), ('Cathy', 'Mon8'), ('Cathy', 'Tue9'), ('Cathy', 'Wed10'),
    ('Cathy', 'Thu11'), ('Cathy', 'Fri12'), ('Cathy', 'Sat13'),
    ('Cathy', 'Sun14'), ('Dan', 'Tue2'), ('Dan', 'Wed3'), ('Dan', 'Fri5'),
    ('Dan', 'Sat6'), ('Dan', 'Mon8'), ('Dan', 'Tue9'), ('Dan', 'Wed10'),
    ('Dan', 'Thu11'), ('Dan', 'Fri12'), ('Dan', 'Sat13'), ('Dan', 'Sun14'),
    ('Ed', 'Mon1'), ('Ed', 'Tue2'), ('Ed', 'Wed3'), ('Ed', 'Thu4'),
    ('Ed', 'Fri5'), ('Ed', 'Sun7'), ('Ed', 'Mon8'), ('Ed', 'Tue9'),
    ('Ed', 'Thu11'), ('Ed', 'Sat13'), ('Ed', 'Sun14'), ('Fred', 'Mon1'),
    ('Fred', 'Tue2'), ('Fred', 'Wed3'), ('Fred', 'Sat6'), ('Fred', 'Mon8'),
    ('Fred', 'Tue9'), ('Fred', 'Fri12'), ('Fred', 'Sat13'), ('Fred', 'Sun14'),
    ('Gu', 'Mon1'), ('Gu', 'Tue2'), ('Gu', 'Wed3'), ('Gu', 'Fri5'),
    ('Gu', 'Sat6'), ('Gu', 'Sun7'), ('Gu', 'Mon8'), ('Gu', 'Tue9'),
    ('Gu', 'Wed10'), ('Gu', 'Thu11'), ('Gu', 'Fri12'), ('Gu', 'Sat13'),
    ('Gu', 'Sun14')
]

m = gp.Model("shift_scheduling")

working = []
slack = []

for i, shift in enumerate(shifts):
    working.append([])
    var1 = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f'shift{i}')
    slack.append(var1)
    for j, worker in enumerate(workers):
        var = m.addVar(vtype=GRB.BINARY, name=f'worker{j}_shift{i}')
        working[-1].append(var)

m.update()

for i, shift in enumerate(shifts):
    m.addConstr(sum(working[i]) + slack[i] == shiftRequirements[shift])
    for j, worker in enumerate(workers):
        if (worker, shift) not in availability:
            m.addConstr(working[i][j] == 0)
max_workload = m.addVar(name="max_workload")
min_workload = m.addVar(name="min_workload")
for j, worker in enumerate(workers):
    workload = sum(working[i][j] for i in range(len(shifts)))
    m.addConstr(max_workload >= workload)
    m.addConstr(min_workload <= workload)


m.setObjectiveN(sum(slack), index=0,priority=2,name='adhbhv')
m.setObjectiveN(max_workload-min_workload,index=1,priority=1,name='hda')

m.optimize()

if m.status == GRB.OPTIMAL:
    print('Solution:')
    print(f'Objective value = {m.objVal}')
    for i, shift in enumerate(shifts):
        for j, worker in enumerate(workers):
            if working[i][j].X > 0.5:
                print(f'{worker} works on {shift}')
else:
    print('No optimal solution found.')


Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 54 rows, 114 columns and 348 nonzeros
Model fingerprint: 0x6d5e47fd
Variable types: 16 continuous, 98 integer (98 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 7e+00]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 31 rows and 31 columns
Presolve time: 0.00s
Presolved: 23 rows, 83 columns, 217 nonzeros
-----

In [40]:
import json
import gurobipy as gp
from gurobipy import GRB
import math
from itertools import combinations
capitals_json = json.load(open('capitals.json'))

capitals = []
coordinates = {}
for state in capitals_json:
   capital = capitals_json[state]['capital']
   capitals.append(capital)
   coordinates[capital] = (float(capitals_json[state]['lat']), float(capitals_json[state]['long']))


def distance(city1, city2):
    c1 = coordinates[city1]
    c2 = coordinates[city2]
    diff = (c1[0]-c2[0], c1[1]-c2[1])
    return math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])
dist = [[0] * len(capitals) for _ in range(len(capitals))]
for i in range(len(capitals)): 
   for j in range(len(capitals)):
      dist[i][j]=distance(capitals[i],capitals[j])


m=gp.Model("tsp")
m.update()
x=[]
status=[]
res=0
for i in range(len(capitals)):
   x.append([])
   var1=m.addVar(lb=1,ub=len(capitals),vtype=GRB.CONTINUOUS,name=f'{i}')
   status.append(var1)
   for j in range(len(capitals)):
      var=m.addVar(vtype=GRB.BINARY,name=f'location{i} to {j}')
      x[-1].append(var)

for i in range(len(capitals)):  
   m.addConstr(sum(x[i][j] for j in range(len(capitals)))==1)
   m.addConstr(sum(x[j][i] for j in range(len(capitals)))==1)
   for j in range(len(capitals)):
      m.addConstr(status[i]-status[j]<=-1+len(capitals)*(x[i][j]-1))
      res+=dist[i][j]*x[i][j]



m.setObjective(res,GRB.MINIMIZE)

m.optimize()

if m.status == GRB.OPTIMAL:
    print('Solution:')
    print(f'Objective value = {m.objVal}')
    
else:
    print('No optimal solution found.')

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads



GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information

In [41]:
import json
import gurobipy as gp
from gurobipy import GRB
import math

# Load the capitals and their coordinates from the JSON file
capitals_json = json.load(open('capitals.json'))

capitals = []
coordinates = {}
for state in capitals_json:
    capital = capitals_json[state]['capital']
    capitals.append(capital)
    coordinates[capital] = (float(capitals_json[state]['lat']), float(capitals_json[state]['long']))

# Function to calculate Euclidean distance between two cities
def distance(city1, city2):
    c1 = coordinates[city1]
    c2 = coordinates[city2]
    diff = (c1[0] - c2[0], c1[1] - c2[1])
    return math.sqrt(diff[0] * diff[0] + diff[1] * diff[1])

# Calculate the distance matrix
dist = [[0] * len(capitals) for _ in range(len(capitals))]
for i in range(len(capitals)):
    for j in range(len(capitals)):
        dist[i][j] = distance(capitals[i], capitals[j])

# Create the Gurobi model
m = gp.Model("tsp")

# Add binary variables for each pair of cities
x = []
for i in range(len(capitals)):
    x.append([])
    for j in range(len(capitals)):
        var = m.addVar(vtype=GRB.BINARY, name=f'location{i} to {j}')
        x[-1].append(var)

# Add degree constraints to ensure each city is visited exactly once
for i in range(len(capitals)):
    m.addConstr(sum(x[i][j] for j in range(len(capitals))) == 1)
    m.addConstr(sum(x[j][i] for j in range(len(capitals))) == 1)

# Objective function to minimize the total distance
res = 0
for i in range(len(capitals)):
    for j in range(len(capitals)):
        res += dist[i][j] * x[i][j]
m.setObjective(res, GRB.MINIMIZE)

# Callback function to eliminate subtours
def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        vals = model.cbGetSolution(model._x)
        selected = gp.tuplelist((i, j) for i in range(len(capitals)) for j in range(len(capitals)) if vals[i][j] > 0.5)
        tour = findsubtour(selected)
        if len(tour) < len(capitals):
            model.cbLazy(gp.quicksum(model._x[i, j] for i in tour for j in tour if i != j) <= len(tour) - 1)

# Function to find the shortest subtour
def findsubtour(selected):
    visited = [False] * len(capitals)
    cycles = []
    lengths = []
    selected = selected[:]
    while selected:
        current = selected[0][0]
        thiscycle = [current]
        while True:
            for i, j in selected:
                if i == current:
                    current = j
                    thiscycle.append(j)
                    selected.remove((i, j))
                    break
            else:
                break
        cycles.append(thiscycle)
        lengths.append(len(thiscycle))
    return cycles[lengths.index(min(lengths))]

# Optimize the model with the subtour elimination callback
m._x = x
m.optimize(subtourelim)

# Print the solution if found
if m.status == GRB.OPTIMAL:
    print('Solution:')
    print(f'Objective value = {m.objVal}')
else:
    print('No optimal solution found.')


Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads


User-callback calls 6, time in user-callback 0.00 sec


GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information